<a href="https://colab.research.google.com/github/shivani1805/AlphaEarth-HackNation-MIT/blob/main/fema_data_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import ee
ee.Authenticate()
ee.Initialize(project="tough-future-477618-d5")

# Load embeddings collection
embeddings = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')

def get_embedding_for_asset(lng, lat, year):
    img = embeddings.filter(ee.Filter.eq('year', year)).first()
    point = ee.Geometry.Point(lng, lat)
    emb_dict = img.reduceRegion(
        reducer = ee.Reducer.first(), geometry = point, scale = 10
    ).getInfo()
    # Convert dict to list/array in consistent order
    vector = [emb_dict[f'A{str(i).zfill(2)}'] for i in range(64)]
    return vector

# Example asset coordinates
# lng, lat = -122.4194, 37.7749  # San Francisco
# vec_pre = get_embedding_for_asset(lng, lat, 2022)
# vec_post = get_embedding_for_asset(lng, lat, 2024)

# Compute difference
# diff_vec = [post - pre for post, pre in zip(vec_post, vec_pre)]

# Now you can feed [vec_pre + diff_vec] into ML model

In [8]:
lng, lat = -122.4194, 37.7749  # San Francisco
vec_pre = get_embedding_for_asset(lng, lat, 2022)

EEException: Image.reduceRegion: Parameter 'image' is required and may not be null.

In [10]:
import requests

BASE_URL = "https://www.ncei.noaa.gov/cdo-web/api/v2"
TOKEN = "sjhjknPHurHbPlQTWcqyeloXPoSBCxSw"

def fetch_noaa(endpoint, params=None):
    """
    Fetches data from NOAA CDO API.
    endpoint: e.g. '/data'
    params: dict of URL parameters (datasetid, locationid, startdate, etc.)
    returns: JSON response
    """
    url = BASE_URL + endpoint
    headers = {'token': TOKEN}
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()  # will throw an error if status != 200
    return response.json()

# Example: list datasets
datasets = fetch_noaa("/datasets", params={'limit': 5})
print(datasets)


{'metadata': {'resultset': {'offset': 1, 'count': 11, 'limit': 5}}, 'results': [{'uid': 'gov.noaa.ncdc:C00861', 'mindate': '1763-01-01', 'maxdate': '2025-11-03', 'name': 'Daily Summaries', 'datacoverage': 1, 'id': 'GHCND'}, {'uid': 'gov.noaa.ncdc:C00946', 'mindate': '1763-01-01', 'maxdate': '2025-09-01', 'name': 'Global Summary of the Month', 'datacoverage': 1, 'id': 'GSOM'}, {'uid': 'gov.noaa.ncdc:C00947', 'mindate': '1763-01-01', 'maxdate': '2025-01-01', 'name': 'Global Summary of the Year', 'datacoverage': 1, 'id': 'GSOY'}, {'uid': 'gov.noaa.ncdc:C00345', 'mindate': '1991-06-05', 'maxdate': '2025-10-12', 'name': 'Weather Radar (Level II)', 'datacoverage': 0.95, 'id': 'NEXRAD2'}, {'uid': 'gov.noaa.ncdc:C00708', 'mindate': '1994-05-20', 'maxdate': '2025-11-05', 'name': 'Weather Radar (Level III)', 'datacoverage': 0.95, 'id': 'NEXRAD3'}]}


In [11]:
# Example query: Storm events in Florida during 2023
storm_data = fetch_noaa(
    "/data",
    params={
        'datasetid': 'STORMEVENTS',      # key dataset for severe weather
        'locationid': 'FIPS:12',         # Florida FIPS code
        'startdate': '2023-01-01',
        'enddate': '2023-12-31',
        'limit': 1000
    }
)
import pandas as pd
df = pd.DataFrame(storm_data.get('results', []))
print(df.head())

HTTPError: 500 Server Error:  for url: https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=STORMEVENTS&locationid=FIPS%3A12&startdate=2023-01-01&enddate=2023-12-31&limit=1000

In [12]:
import requests
import pandas as pd

# Example: Gage height for station 01646500 (Potomac River)
url = "https://waterservices.usgs.gov/nwis/dv/"
params = {
    "format": "json",
    "sites": "01646500",
    "startDT": "2021-07-01",
    "endDT": "2021-07-10",
    "parameterCd": "00065"  # Gage height
}

response = requests.get(url, params=params)
data = response.json()["value"]["timeSeries"][0]["values"][0]["value"]

df = pd.DataFrame(data)
print(df.head())


IndexError: list index out of range

In [20]:
import requests
import pandas as pd

BASE_URL = "https://www.fema.gov/api/open/v2/FimaNfipClaims"

def query_nfip_claims(filter_str=None, top=1000, skip=0, select=None):
    params = {
        "$top": top,
        "$skip": skip
    }
    if filter_str:
        params["$filter"] = filter_str
    if select:
        params["$select"] = select
    response = requests.get(BASE_URL, params=params)
    response.raise_for_status()
    data = response.json()
    results = data.get("FimaNfipClaims", [])
    return pd.DataFrame(results)

# Example usage: get the first 1000 claims in state FL
df = query_nfip_claims(filter_str="state eq 'FL'", top=5)
print(df.head())




   agricultureStructureIndicator                  asOfDate  \
0                          False  2025-09-09T00:00:00.000Z   
1                          False  2025-09-09T00:00:00.000Z   
2                          False  2025-09-09T00:00:00.000Z   
3                          False  2025-09-09T00:00:00.000Z   
4                          False  2025-09-09T00:00:00.000Z   

   basementEnclosureCrawlspaceType  policyCount crsClassificationCode  \
0                              NaN            1                  None   
1                              0.0            1                  None   
2                              NaN            1                  None   
3                              NaN            1                  None   
4                              NaN            1                  None   

                 dateOfLoss  elevatedBuildingIndicator  \
0  2009-03-19T00:00:00.000Z                       True   
1  2022-09-28T00:00:00.000Z                      False   
2  2022-09-28T

In [21]:
df = query_nfip_claims(
    filter_str="state eq 'HI'",
    top=None,
    select="state,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,yearOfLoss,floodEvent"
)
print(df.head())

# Fill missing values with 0
df["amountPaidOnBuildingClaim"] = df["amountPaidOnBuildingClaim"].fillna(0)
df["amountPaidOnContentsClaim"] = df["amountPaidOnContentsClaim"].fillna(0)

# Compute total
df["totalPaid"] = df["amountPaidOnBuildingClaim"] + df["amountPaidOnContentsClaim"]
print(df.head())

  state  amountPaidOnBuildingClaim  amountPaidOnContentsClaim  yearOfLoss  \
0    HI                        NaN                        NaN        2005   
1    HI                   54818.25                    9934.11        1983   
2    HI                   19892.31                       0.00        1982   
3    HI                    1275.42                       0.00        1985   
4    HI                    1414.67                       0.00        2024   

  floodEvent  
0       None  
1       None  
2   Flooding  
3       None  
4       None  
  state  amountPaidOnBuildingClaim  amountPaidOnContentsClaim  yearOfLoss  \
0    HI                       0.00                       0.00        2005   
1    HI                   54818.25                    9934.11        1983   
2    HI                   19892.31                       0.00        1982   
3    HI                    1275.42                       0.00        1985   
4    HI                    1414.67                       0.00 

In [23]:
# df = query_nfip_claims(
#     filter_str="state eq 'HI'",
#     top=None,
#     select="state,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,yearOfLoss,floodEvent,reportedZipCode"
# )
df = query_nfip_claims(
    filter_str="",
    top=None,
    select="state,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,yearOfLoss,floodEvent,reportedZipCode"
)
print(df.head())

# Fill missing values with 0
df["amountPaidOnBuildingClaim"] = df["amountPaidOnBuildingClaim"].fillna(0)
df["amountPaidOnContentsClaim"] = df["amountPaidOnContentsClaim"].fillna(0)

# Compute total
df["totalPaid"] = df["amountPaidOnBuildingClaim"] + df["amountPaidOnContentsClaim"]
print(df.head())

  state  amountPaidOnBuildingClaim  amountPaidOnContentsClaim  yearOfLoss  \
0    TX                   50000.00                   20000.00        2017   
1    VA                   12843.93                    4428.58        2003   
2    FL                        NaN                        NaN        2009   
3    LA                        NaN                        NaN        2005   
4    LA                     206.05                       0.00        1995   

          floodEvent reportedZipCode  
0   Hurricane Harvey           77429  
1   Hurricane Isabel           23508  
2               None           33433  
3  Hurricane Katrina           70003  
4           Flooding           70094  
  state  amountPaidOnBuildingClaim  amountPaidOnContentsClaim  yearOfLoss  \
0    TX                   50000.00                   20000.00        2017   
1    VA                   12843.93                    4428.58        2003   
2    FL                       0.00                       0.00        200

In [27]:
BASE_URL = "https://www.fema.gov/api/open/v2/FimaNfipClaims"

def query_nfip_claims(filter_str=None, top=1000, skip=0, select=None):
    params = {
        "$top": top,
        "$skip": skip
    }
    if filter_str:
        params["$filter"] = filter_str
    if select:
        params["$select"] = select
    response = requests.get(BASE_URL, params=params)
    response.raise_for_status()
    data = response.json()
    results = data.get("FimaNfipClaims", [])
    return pd.DataFrame(results)

def get_payment_amounts(zip_code, date_of_loss):
  df = query_nfip_claims(
      filter_str=f"reportedZipCode eq '{zip_code}' and yearOfLoss eq {date_of_loss}",
      top=None,
      select="state,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,yearOfLoss,floodEvent"
  )
  df["amountPaidOnBuildingClaim"] = df["amountPaidOnBuildingClaim"].fillna(0)
  df["amountPaidOnContentsClaim"] = df["amountPaidOnContentsClaim"].fillna(0)

  # Compute total
  df["totalPaid"] = df["amountPaidOnBuildingClaim"] + df["amountPaidOnContentsClaim"]
  return df

df = get_payment_amounts(77429, 2017)
print(df)


    state  amountPaidOnBuildingClaim  amountPaidOnContentsClaim  yearOfLoss  \
0      TX                   50000.00                   20000.00        2017   
1      TX                  174042.00                   46958.00        2017   
2      TX                  103420.65                    7789.53        2017   
3      TX                   18899.67                       0.00        2017   
4      TX                  123774.31                   44967.77        2017   
..    ...                        ...                        ...         ...   
995    TX                    5214.34                       0.00        2017   
996    TX                  110434.90                   29024.69        2017   
997    TX                   97659.02                   87583.63        2017   
998    TX                       0.00                       0.00        2017   
999    TX                  150530.25                   43568.34        2017   

           floodEvent  totalPaid  
0    Hurricane H

In [ ]:
def get_all_flood_event_counts():
    """Retrieve all unique floodEvent types and their counts across the dataset."""
    all_data = []
    top = 1000
    skip = 0

    while True:
        df = query_nfip_claims(top=top, skip=skip, select="floodEvent")
        if df.empty:
            break
        all_data.append(df)
        skip += top

    full_df = pd.concat(all_data, ignore_index=True)
    counts = full_df["floodEvent"].value_counts(dropna=False)
    return counts

# Example usage
flood_event_counts = get_all_flood_event_counts()
print(flood_event_counts)